# Metody Obliczeniowe w Nauce i Technice Laboratorium 5
## Równania nieliniowe
### Błażej Kustra

### Funkcje do testów

### 1. Metoda bisekcji
Napisz funkcję realizującą metodę bisekcji dla danej funkcji f w oparciu o arytmetykę o zmiennej precyzji (mpmath.mpf). Funkcja przyjmuje następujące argumenty:
- Minimalną precyzję obliczeń (liczba cyfr znaczących) 
- Krańce przedziału
- Błąd bezwzględny obliczeń1
Funkcja ma zwracać wyznaczone miejsce zerowe, wyliczoną wartość w miejscu zerowym oraz liczbę iteracji potrzebną do uzyskania określonej dokładności. Przetestuj działanie metody dla funkcji podanych na początku instrukcji. Jaka liczba iteracji jest potrzebna do uzyskania bezwzględnej dokładności rzędu: $10^{−7}, 10^{−15}, 10^{−33}$? W jaki sposób możemy obliczyć k pierwszych dodatnich pierwiastków funkcji $f_1(x)$?

Można pokazać, że uzyskanie bezwzględnej dokładności ε wymaga $n = \lceil {\frac{log \frac{b-a}{ε}}{log 2} }\rceil$ iteracji

### 2. Metoda Newtona
Napisz funkcję realizującą metodę Newtona w oparciu o arytmetykę o zmiennej precyzji (mpmath.mpf). Funkcja ma wykorzystywać dwa kryteria stopu:
- maksymalną liczbę iteracji
- moduł różnicy kolejnych przybliżeń mniejszy od danej wartości ε

Oprócz przybliżonej wartości pierwiastka funkcja ma zwrócić liczbę iteracji potrzebną do uzyskania określonej dokładności ε. Przetestuj działanie metody dla funkcji podanych na początku instrukcji (dodatkowo dostępne pochodne tych funkcji). Porównaj zbieżność metody ze zbieżnością uzyskaną dla metody bisekcji.

### 3. Metoda siecznych
Napisz funkcję realizującą metodę siecznych w oparciu o arytmetykę o zmiennej precyzji (mpmath.mpf). Funkcja powinna stosować te same kryteria stopu co funkcja realizująca metodę Newtona. Przetestuj działanie metody dla funkcji podanych na początku instrukcji. Porównaj zbieżność metody ze zbieżnością uzyskaną dla metody bisekcji oraz metody Newtona.